In [1]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import json 


In [2]:
base_path = 'E:\\indoor-location-navigation\\'

In [3]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv(base_path + 'sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [4]:
# get only the wifi bssid that occur over 1000 times(this number can be experimented with)
# these will be the only ones used when constructing features
bssid = dict()

In [21]:


for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train\\'+building+'\\*')))
    print(building)
    wifi = list()
    for folder in folders:
        print(folder)
        floor = floor_map[folder.split('\\')[-1]]
        print(floor)
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            with open(file, encoding="utf8") as f:
                txt = f.readlines()
                for e, line in enumerate(txt):
                    tmp = line.strip().split()
                    if tmp[1] == "TYPE_WIFI":
                        wifi.append(tmp)
    df = pd.DataFrame(wifi)
    #top_bssid = df[3].value_counts().iloc[:500].index.tolist()
    value_counts = df[3].value_counts()
    top_bssid = value_counts[value_counts > 0].index.tolist()
    print(len(top_bssid))
    bssid[building] = top_bssid
    del df
    del wifi
    gc.collect()

5a0546857ecc773753327266
E:\indoor-location-navigation\train\5a0546857ecc773753327266\B1
-1
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F1
0
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F2
1
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F3
2
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F4
3
3397
5c3c44b80379370013e0fd2b
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\B1
-1
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F1
0
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F2
1
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F3
2
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F4
3
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F5
4
3063
5d27075f03f801723c2e360f
E:\indoor-location-navigation\train\5d27075f03f801723c2e360f\B1
-1
E:\indoor-location-navigation\train\5d27075f03f801723c2e360f\F1
0
E:\indoor-location-navigation\train\5d27075f03f801723c

3535
5da958dd46f8266d0737457b
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\B1
-1
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F1
0
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F2
1
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F3
2
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F4
3
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F5
4
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F6
5
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F7
6
3499
5dbc1d84c1eb61796cf7c010
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\B1
-1
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F2
1
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F3
2
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F4
3
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F5
4
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F6
5
E:\indoor-loca

In [22]:
with open("bssid_1000.json", "w") as f:
    json.dump(bssid, f)

In [4]:
with open("bssid_1000.json") as f:
    bssid = json.load(f)

In [5]:
# generate all the training data 
building_dfs = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train/', building +'/*')))
    dfs = list()
    index = sorted(bssid[building])
    print(building)
    for folder in folders:
        floor = floor_map[folder.split('\\')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            wifi = list()
            waypoint = list()
            magnetic = list()
            accelerometer = list()
            with open(file, encoding="utf8") as f:
                txt = f.readlines()
            for line in txt:
                line = line.strip().split()
                if line[1] == "TYPE_WAYPOINT":
                    waypoint.append(line)
                if line[1] == "TYPE_WIFI":
                    wifi.append(line)
                if line[1] == "TYPE_MAGNETIC_FIELD":
                    magnetic.append(line)
                if line[1] == "TYPE_ACCELEROMETER":
                    accelerometer.append(line)
                

            df = pd.DataFrame(np.array(wifi))  
            df_magnet = pd.DataFrame(np.array(magnetic))
            df_magnet[0] = pd.to_numeric(df_magnet[0])
            df_magnet[2] = pd.to_numeric(df_magnet[2])
            df_magnet[3] = pd.to_numeric(df_magnet[3])
            df_magnet[4] = pd.to_numeric(df_magnet[4])
            
            df_accelerometer = pd.DataFrame(np.array(accelerometer))
            df_accelerometer[0] = pd.to_numeric(df_accelerometer[0])
            df_accelerometer[2] = pd.to_numeric(df_accelerometer[2])
            df_accelerometer[3] = pd.to_numeric(df_accelerometer[3])
            df_accelerometer[4] = pd.to_numeric(df_accelerometer[4])
            
            begining_timestamp = 0

            # generate a feature, and label for each wifi block
            for gid, g in df.groupby(0):
                dists = list()
                gid_int = int(gid)
                for e, k in enumerate(waypoint):
                    dist = abs(gid_int - int(k[0]))
                    dists.append(dist)
                nearest_wp_index = np.argmin(dists)
                
                
                x_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< gid_int), 2].mean()
                y_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< gid_int), 3].mean()
                z_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< gid_int), 4].mean()
                
                x_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< gid_int), 2].mean()
                y_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< gid_int), 3].mean()
                z_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< gid_int), 4].mean()
                
                begining_timestamp = gid_int
                
                g = g.drop_duplicates(subset=3)
                tmp = g.iloc[:,3:5]
                feat = tmp.set_index(3).reindex(index).T
                
                
                feat['x_accel']=x_accel
                feat['y_accel']=y_accel
                feat['z_accel']=z_accel
                
                feat['x_magnet']=x_magnet
                feat['y_magnet']=y_magnet
                feat['z_magnet']=z_magnet
                feat["x"] = float(waypoint[nearest_wp_index][2])
                feat["y"] = float(waypoint[nearest_wp_index][3])
                feat["f"] = floor
                feat["path"] = file.split('/')[-1].split('.')[0] # useful for crossvalidation
                dfs.append(feat)
                
    building_df = pd.concat(dfs)
    building_dfs[building] = df
    building_df.to_csv(building+"_1000_train.csv")

5a0546857ecc773753327266
5c3c44b80379370013e0fd2b
5d27075f03f801723c2e360f
5d27096c03f801723c31e5e0
5d27097f03f801723c320d97
5d27099f03f801723c32511d
5d2709a003f801723c3251bf
5d2709b303f801723c327472
5d2709bb03f801723c32852c
5d2709c303f801723c3299ee
5d2709d403f801723c32bd39
5d2709e003f801723c32d896
5da138274db8ce0c98bbd3d2
5da1382d4db8ce0c98bbe92e
5da138314db8ce0c98bbf3a0
5da138364db8ce0c98bc00f1
5da1383b4db8ce0c98bc11ab
5da138754db8ce0c98bca82f
5da138764db8ce0c98bcaa46
5da1389e4db8ce0c98bd0547
5da138b74db8ce0c98bd4774
5da958dd46f8266d0737457b
5dbc1d84c1eb61796cf7c010
5dc8cea7659e181adb076a3f


In [8]:
ssubm_building_g = ssubm_df.groupby(0) #Grouped by building
feature_dict = dict()

for gid0, g0 in ssubm_building_g:
    index = sorted(bssid[g0.iloc[0,0]])
    feats = list()
    print(gid0, 'gid0')
#     print(g0, 'g0')
    #print()
    for gid,g in g0.groupby(1): #Grouping by path
        # get all wifi time locations, 
        #print(gid, 'gid')
        #print(g, 'g')
#       print()
        with open(os.path.join(base_path, 'test\\' + g.iloc[0,1] + '.txt'), encoding="utf8") as f:
            txt = f.readlines()

        wifi = list()
        magnetic = list()
        accelerometer = list()

        for line in txt:
            line = line.strip().split()
            if line[1] == "TYPE_WIFI":
                wifi.append(line)
            if line[1] == "TYPE_MAGNETIC_FIELD":
                magnetic.append(line)
            if line[1] == "TYPE_ACCELEROMETER":
                accelerometer.append(line)

        wifi_df = pd.DataFrame(wifi) #All wifi points for certain path
        df_magnet = pd.DataFrame(np.array(magnetic))
        df_magnet[0] = pd.to_numeric(df_magnet[0])
        df_magnet[2] = pd.to_numeric(df_magnet[2])
        df_magnet[3] = pd.to_numeric(df_magnet[3])
        df_magnet[4] = pd.to_numeric(df_magnet[4])
        
        
        df_accelerometer = pd.DataFrame(np.array(accelerometer))
        df_accelerometer[0] = pd.to_numeric(df_accelerometer[0])
        df_accelerometer[2] = pd.to_numeric(df_accelerometer[2])
        df_accelerometer[3] = pd.to_numeric(df_accelerometer[3])
        df_accelerometer[4] = pd.to_numeric(df_accelerometer[4])
        
        begining_timestamp = 0
        
        wifi_points = pd.DataFrame(wifi_df.groupby(0).count().index.tolist()) #Timestamps for wifi for certain path
        magnetic_points = pd.DataFrame(df_magnet.groupby(0).count().index.tolist()) #Timestamps for magnetometer for certain path
        
        #print(wifi_points)
        begining_timestamp=0
        for timepoint in g.iloc[:,2].tolist():
            timepoint_int = int(timepoint)
            deltas = (wifi_points.astype(int) - timepoint_int).abs()
            min_delta_idx = deltas.values.argmin()
            wifi_block_timestamp = wifi_points.iloc[min_delta_idx].values[0]
            
            
            
            x_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< timepoint_int), 2].mean()
            y_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< timepoint_int), 3].mean()
            z_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< timepoint_int), 4].mean()
            
            
            x_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< timepoint_int), 2].mean()
            y_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< timepoint_int), 3].mean()
            z_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< timepoint_int), 4].mean()
                
            
            if x_magnet !=x_magnet:
                x_magnet = df_magnet.iloc[0, 2]
                y_magnet = df_magnet.iloc[0, 3]
                z_magnet = df_magnet.iloc[0, 4]
            if x_accel !=x_accel:
                x_accel = df_accelerometer.iloc[0, 2]
                y_accel = df_accelerometer.iloc[0, 3]
                z_accel = df_accelerometer.iloc[0, 4]
                #print(x_magnet, ' assigned')
            begining_timestamp = timepoint_int
            wifi_block = wifi_df[wifi_df[0] == wifi_block_timestamp].drop_duplicates(subset=3)
            feat = wifi_block.set_index(3)[4].reindex(index)
            
            feat['x_accel']=x_accel
            feat['y_accel']=y_accel
            feat['z_accel']=z_accel
            
            feat['x_magnet']=x_magnet
            feat['y_magnet']=y_magnet
            feat['z_magnet']=z_magnet
            
            feat['site_path_timestamp'] = g.iloc[0,0] + "_" + g.iloc[0,1] + "_" + timepoint
            feats.append(feat)
    feature_df = pd.concat(feats, axis=1).T
    feature_df.to_csv(gid0+"_1000_test.csv")
    feature_dict[gid0] = feature_df

5a0546857ecc773753327266 gid0
5c3c44b80379370013e0fd2b gid0
5d27075f03f801723c2e360f gid0
5d27096c03f801723c31e5e0 gid0
5d27097f03f801723c320d97 gid0
5d27099f03f801723c32511d gid0
5d2709a003f801723c3251bf gid0
5d2709b303f801723c327472 gid0
5d2709bb03f801723c32852c gid0
5d2709c303f801723c3299ee gid0
5d2709d403f801723c32bd39 gid0
5d2709e003f801723c32d896 gid0
5da138274db8ce0c98bbd3d2 gid0
5da1382d4db8ce0c98bbe92e gid0
5da138314db8ce0c98bbf3a0 gid0
5da138364db8ce0c98bc00f1 gid0
5da1383b4db8ce0c98bc11ab gid0
5da138754db8ce0c98bca82f gid0
5da138764db8ce0c98bcaa46 gid0
5da1389e4db8ce0c98bd0547 gid0
5da138b74db8ce0c98bd4774 gid0
5da958dd46f8266d0737457b gid0
5dbc1d84c1eb61796cf7c010 gid0
5dc8cea7659e181adb076a3f gid0


In [18]:
import xgboost as xgb

In [19]:
feature_path = 'C:\\Users\\user\\Documents\\LightHouseLabs\\W09\\'

In [20]:
train_files = sorted(glob.glob(os.path.join(feature_path, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_path, '*_test.csv')))

In [21]:
predictions = list()

for e, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    test_data = pd.read_csv(test_files[e], index_col=0)

    # simple grid search for tuning using path and groupkfold cv
    #for i in [127]:
    #    for n in [75]:
    #        modely = lgb.LGBMRegressor(
    #        n_estimators=n, num_leaves=i, n_jobs=1)
    #        gkf = model_selection.GroupKFold()
    #        scores = model_selection.cross_val_score(
    #            modely, x_train, y_trainy, scoring='neg_mean_squared_error',
    #            cv=gkf, groups=data.iloc[:,-1], n_jobs=5)
    #        print(i, n, l, scores, scores.mean())
    
    data = data.sample(frac=1)
    x_train = data.iloc[:int(len(data)*0.70),:-4]
    y_trainy = data.iloc[:int(len(data)*0.70),-3]
    y_trainx = data.iloc[:int(len(data)*0.70),-4]
    y_trainf = data.iloc[:int(len(data)*0.70),-2] + 2 #To make floors classes from 0 to 10
    
    x_test = data.iloc[int(len(data)*0.70):,:-4]
    y_testy = data.iloc[int(len(data)*0.70):,-3]
    y_testx = data.iloc[int(len(data)*0.70):,-4]
    y_testf = data.iloc[int(len(data)*0.70):,-2] + 2 #To make floors classes from 0 to 10
    
    
    dtrainX = xgb.DMatrix(x_train, label=y_trainx)
    dtestX = xgb.DMatrix(x_test, label=y_testx)
    
    dtrainY = xgb.DMatrix(x_train, label=y_trainy)
    dtestY = xgb.DMatrix(x_test, label=y_testy)
    
    dtrainF = xgb.DMatrix(x_train, label=y_trainf)
    dtestF = xgb.DMatrix(x_test, label=y_testf)
    
    paramsXY = {
    # Parameters that we are going to tune.
    'max_depth':90,
    'min_child_weight': 0.5,
    'eta':.075,
    'subsample': 1,
    'colsample_bytree': 0.3,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    'booster':'gbtree',
    'random_state':42,
    'verbosity':1,
    'n_jobs':-1
    }
    
    paramsF = {
    # Parameters that we are going to tune.
    'max_depth':90,
    'min_child_weight': 0.5,
    'eta':.075,
    'subsample': 1,
    'colsample_bytree': 0.3,
    # Other parameters
    'objective':'multi:softmax',
    'eval_metric':'merror',
    'num_class':11,
    'booster':'gbtree',
    'random_state':42,
    'verbosity':1,
    'n_jobs':-1
    }
    
    num_boost_round=5000
    
    xgb_rX = xgb.train(
    paramsXY,
    dtrainX,
    num_boost_round=num_boost_round,
    evals=[(dtestX, "Test")],
    early_stopping_rounds=30
    )
    
    xgb_rY = xgb.train(
    paramsXY,
    dtrainY,
    num_boost_round=num_boost_round,
    evals=[(dtestY, "Test")],
    early_stopping_rounds=30
    )
    
    xgb_cF = xgb.train(
    paramsF,
    dtrainF,
    num_boost_round=num_boost_round,
    evals=[(dtestF, "Test")],
    early_stopping_rounds=30
    )
    
#     modely = lgb.LGBMRegressor(
#         n_estimators=125, num_leaves=90)
#     modely.fit(x_train, y_trainy)

#     modelx = lgb.LGBMRegressor(
#         n_estimators=125, num_leaves=90)
#     modelx.fit(x_train, y_trainx)

#     modelf = lgb.LGBMClassifier(
#         n_estimators=125, num_leaves=90)
#     modelf.fit(x_train, y_trainf)
    
    test_predsx = xgb_rX.predict(xgb.DMatrix(test_data.iloc[:,:-1]))
    test_predsy = xgb_rY.predict(xgb.DMatrix(test_data.iloc[:,:-1]))
    test_predsf = xgb_cF.predict(xgb.DMatrix(test_data.iloc[:,:-1]))
    
    test_preds = pd.DataFrame(np.stack((test_predsf, test_predsx, test_predsy))).T
    test_preds.columns = ssubm.columns[1:]
    test_preds.index = test_data["site_path_timestamp"]
    
    test_preds["floor"] = test_preds["floor"].astype(int)-2 #Returning floor classes back to original
    
    #Post-processing should go here. Idea is to snap to waypoint if the prediction is to close.
    #*************************
    #Generating dataframe with unique waypoints
    
    #data.drop_duplicates(subset=['x', 'y', 'f'], inplace=True)
    
    #*********End of post processing**********
    
    
    
    
    predictions.append(test_preds)

[0]	Test-rmse:105.82137
[1]	Test-rmse:98.05155
[2]	Test-rmse:90.84740
[3]	Test-rmse:84.18204
[4]	Test-rmse:77.99910
[5]	Test-rmse:72.27959
[6]	Test-rmse:66.98058
[7]	Test-rmse:62.08457
[8]	Test-rmse:57.54090
[9]	Test-rmse:53.33920
[10]	Test-rmse:49.44924
[11]	Test-rmse:45.84869
[12]	Test-rmse:42.50803
[13]	Test-rmse:39.41675
[14]	Test-rmse:36.56720
[15]	Test-rmse:33.93527
[16]	Test-rmse:31.47870
[17]	Test-rmse:29.22626
[18]	Test-rmse:27.14175
[19]	Test-rmse:25.21120
[20]	Test-rmse:23.43171
[21]	Test-rmse:21.76944
[22]	Test-rmse:20.25193
[23]	Test-rmse:18.84713
[24]	Test-rmse:17.55767
[25]	Test-rmse:16.35963
[26]	Test-rmse:15.25595
[27]	Test-rmse:14.23847
[28]	Test-rmse:13.30481
[29]	Test-rmse:12.44557
[30]	Test-rmse:11.64658
[31]	Test-rmse:10.91365
[32]	Test-rmse:10.24309
[33]	Test-rmse:9.62666
[34]	Test-rmse:9.06751
[35]	Test-rmse:8.55255
[36]	Test-rmse:8.07320
[37]	Test-rmse:7.63309
[38]	Test-rmse:7.23541
[39]	Test-rmse:6.87177
[40]	Test-rmse:6.54630
[41]	Test-rmse:6.24109
[42]	Test-

[345]	Test-rmse:3.48600
[346]	Test-rmse:3.48600
[347]	Test-rmse:3.48600
[348]	Test-rmse:3.48600
[349]	Test-rmse:3.48600
[350]	Test-rmse:3.48600
[351]	Test-rmse:3.48600
[352]	Test-rmse:3.48600
[353]	Test-rmse:3.48600
[354]	Test-rmse:3.48600
[355]	Test-rmse:3.48600
[356]	Test-rmse:3.48600
[357]	Test-rmse:3.48600
[358]	Test-rmse:3.48600
[359]	Test-rmse:3.48600
[360]	Test-rmse:3.48600
[361]	Test-rmse:3.48600
[362]	Test-rmse:3.48600
[363]	Test-rmse:3.48600
[364]	Test-rmse:3.48600
[365]	Test-rmse:3.48600
[366]	Test-rmse:3.48600
[367]	Test-rmse:3.48600
[368]	Test-rmse:3.48600
[369]	Test-rmse:3.48600
[370]	Test-rmse:3.48600
[371]	Test-rmse:3.48600
[0]	Test-rmse:97.06679
[1]	Test-rmse:89.86015
[2]	Test-rmse:83.20135
[3]	Test-rmse:77.03670
[4]	Test-rmse:71.32954
[5]	Test-rmse:66.04594
[6]	Test-rmse:61.17381
[7]	Test-rmse:56.67412
[8]	Test-rmse:52.48076
[9]	Test-rmse:48.62352
[10]	Test-rmse:45.04654
[11]	Test-rmse:41.74232
[12]	Test-rmse:38.67905
[13]	Test-rmse:35.85509
[14]	Test-rmse:33.24058
[1

[12]	Test-merror:0.00000
[13]	Test-merror:0.00000
[14]	Test-merror:0.00000
[15]	Test-merror:0.00000
[16]	Test-merror:0.00000
[17]	Test-merror:0.00000
[18]	Test-merror:0.00000
[19]	Test-merror:0.00000
[20]	Test-merror:0.00000
[21]	Test-merror:0.00000
[22]	Test-merror:0.00000
[23]	Test-merror:0.00000
[24]	Test-merror:0.00000
[25]	Test-merror:0.00000
[26]	Test-merror:0.00000
[27]	Test-merror:0.00000
[28]	Test-merror:0.00000
[29]	Test-merror:0.00000
[30]	Test-merror:0.00000
[31]	Test-merror:0.00000
[32]	Test-merror:0.00000
[0]	Test-rmse:217.56633
[1]	Test-rmse:201.36450
[2]	Test-rmse:186.40974
[3]	Test-rmse:172.55374
[4]	Test-rmse:159.72914
[5]	Test-rmse:147.87660
[6]	Test-rmse:136.90683
[7]	Test-rmse:126.73914
[8]	Test-rmse:117.33315
[9]	Test-rmse:108.63479
[10]	Test-rmse:100.58651
[11]	Test-rmse:93.15514
[12]	Test-rmse:86.28114
[13]	Test-rmse:79.91661
[14]	Test-rmse:74.04800
[15]	Test-rmse:68.62221
[16]	Test-rmse:63.56820
[17]	Test-rmse:58.92274
[18]	Test-rmse:54.62082
[19]	Test-rmse:50.

[322]	Test-rmse:4.97287
[323]	Test-rmse:4.97287
[324]	Test-rmse:4.97287
[325]	Test-rmse:4.97287
[326]	Test-rmse:4.97287
[327]	Test-rmse:4.97287
[328]	Test-rmse:4.97287
[329]	Test-rmse:4.97287
[330]	Test-rmse:4.97287
[331]	Test-rmse:4.97287
[332]	Test-rmse:4.97287
[333]	Test-rmse:4.97287
[334]	Test-rmse:4.97287
[0]	Test-rmse:97.94223
[1]	Test-rmse:90.71282
[2]	Test-rmse:84.00743
[3]	Test-rmse:77.80500
[4]	Test-rmse:72.06110
[5]	Test-rmse:66.74470
[6]	Test-rmse:61.83224
[7]	Test-rmse:57.29203
[8]	Test-rmse:53.09020
[9]	Test-rmse:49.21019
[10]	Test-rmse:45.61438
[11]	Test-rmse:42.29820
[12]	Test-rmse:39.22358
[13]	Test-rmse:36.38267
[14]	Test-rmse:33.75261
[15]	Test-rmse:31.34072
[16]	Test-rmse:29.09586
[17]	Test-rmse:27.03224
[18]	Test-rmse:25.11774
[19]	Test-rmse:23.34905
[20]	Test-rmse:21.71559
[21]	Test-rmse:20.20481
[22]	Test-rmse:18.81422
[23]	Test-rmse:17.52889
[24]	Test-rmse:16.34911
[25]	Test-rmse:15.25927
[26]	Test-rmse:14.25786
[27]	Test-rmse:13.33317
[28]	Test-rmse:12.48055
[2

[9]	Test-merror:0.00000
[10]	Test-merror:0.00000
[11]	Test-merror:0.00000
[12]	Test-merror:0.00000
[13]	Test-merror:0.00000
[14]	Test-merror:0.00000
[15]	Test-merror:0.00000
[16]	Test-merror:0.00000
[17]	Test-merror:0.00000
[18]	Test-merror:0.00000
[19]	Test-merror:0.00000
[20]	Test-merror:0.00000
[21]	Test-merror:0.00034
[22]	Test-merror:0.00000
[23]	Test-merror:0.00000
[24]	Test-merror:0.00000
[25]	Test-merror:0.00000
[26]	Test-merror:0.00000
[27]	Test-merror:0.00000
[28]	Test-merror:0.00000
[29]	Test-merror:0.00000
[30]	Test-merror:0.00000
[31]	Test-merror:0.00000
[0]	Test-rmse:159.32535
[1]	Test-rmse:147.46808
[2]	Test-rmse:136.50931
[3]	Test-rmse:126.37684
[4]	Test-rmse:116.99664
[5]	Test-rmse:108.30790
[6]	Test-rmse:100.28213
[7]	Test-rmse:92.85427
[8]	Test-rmse:85.98315
[9]	Test-rmse:79.62682
[10]	Test-rmse:73.74547
[11]	Test-rmse:68.30090
[12]	Test-rmse:63.26240
[13]	Test-rmse:58.60992
[14]	Test-rmse:54.29382
[15]	Test-rmse:50.31462
[16]	Test-rmse:46.63598
[17]	Test-rmse:43.228

...

[297]	Test-rmse:3.12678
[298]	Test-rmse:3.12678
[299]	Test-rmse:3.12678
[300]	Test-rmse:3.12678
[301]	Test-rmse:3.12678
[0]	Test-merror:0.00792
[1]	Test-merror:0.00110
[2]	Test-merror:0.00066
[3]	Test-merror:0.00066
[4]	Test-merror:0.00044
[5]	Test-merror:0.00044
[6]	Test-merror:0.00044
[7]	Test-merror:0.00044
[8]	Test-merror:0.00044
[9]	Test-merror:0.00044
[10]	Test-merror:0.00044
[11]	Test-merror:0.00044
[12]	Test-merror:0.00044
[13]	Test-merror:0.00044
[14]	Test-merror:0.00044
[15]	Test-merror:0.00044
[16]	Test-merror:0.00044
[17]	Test-merror:0.00022
[18]	Test-merror:0.00044
[19]	Test-merror:0.00022
[20]	Test-merror:0.00022
[21]	Test-merror:0.00022
[22]	Test-merror:0.00022
[23]	Test-merror:0.00022
[24]	Test-merror:0.00022
[25]	Test-merror:0.00022
[26]	Test-merror:0.00022
[27]	Test-merror:0.00022
[28]	Test-merror:0.00022
[29]	Test-merror:0.00022
[30]	Test-merror:0.00000
[31]	Test-merror:0.00022
[32]	Test-merror:0.00000
[33]	Test-merror:0.00000
[34]	Test-merror:0.00000
[35]	Test-merro

[278]	Test-rmse:4.21171
[279]	Test-rmse:4.21171
[280]	Test-rmse:4.21171
[281]	Test-rmse:4.21171
[282]	Test-rmse:4.21171
[283]	Test-rmse:4.21171
[284]	Test-rmse:4.21171
[285]	Test-rmse:4.21171
[286]	Test-rmse:4.21171
[287]	Test-rmse:4.21171
[288]	Test-rmse:4.21171
[289]	Test-rmse:4.21171
[290]	Test-rmse:4.21171
[291]	Test-rmse:4.21171
[292]	Test-rmse:4.21171
[293]	Test-rmse:4.21171
[294]	Test-rmse:4.21171
[295]	Test-rmse:4.21171
[296]	Test-rmse:4.21171
[297]	Test-rmse:4.21171
[298]	Test-rmse:4.21171
[299]	Test-rmse:4.21171
[300]	Test-rmse:4.21171
[301]	Test-rmse:4.21171
[302]	Test-rmse:4.21171
[303]	Test-rmse:4.21171
[304]	Test-rmse:4.21171
[305]	Test-rmse:4.21171
[306]	Test-rmse:4.21171
[307]	Test-rmse:4.21171
[308]	Test-rmse:4.21171
[309]	Test-rmse:4.21171
[310]	Test-rmse:4.21171
[311]	Test-rmse:4.21171
[312]	Test-rmse:4.21171
[313]	Test-rmse:4.21171
[314]	Test-rmse:4.21171
[315]	Test-rmse:4.21171
[316]	Test-rmse:4.21171
[317]	Test-rmse:4.21171
[318]	Test-rmse:4.21171
[319]	Test-rmse:

[295]	Test-rmse:4.33899
[296]	Test-rmse:4.33899
[297]	Test-rmse:4.33899
[298]	Test-rmse:4.33899
[299]	Test-rmse:4.33899
[300]	Test-rmse:4.33899
[301]	Test-rmse:4.33899
[302]	Test-rmse:4.33899
[303]	Test-rmse:4.33899
[304]	Test-rmse:4.33899
[305]	Test-rmse:4.33899
[306]	Test-rmse:4.33899
[307]	Test-rmse:4.33899
[308]	Test-rmse:4.33899
[309]	Test-rmse:4.33899
[310]	Test-rmse:4.33899
[311]	Test-rmse:4.33898
[312]	Test-rmse:4.33898
[313]	Test-rmse:4.33898
[314]	Test-rmse:4.33898
[315]	Test-rmse:4.33898
[316]	Test-rmse:4.33898
[317]	Test-rmse:4.33898
[318]	Test-rmse:4.33898
[319]	Test-rmse:4.33898
[320]	Test-rmse:4.33898
[321]	Test-rmse:4.33898
[322]	Test-rmse:4.33898
[323]	Test-rmse:4.33898
[324]	Test-rmse:4.33898
[325]	Test-rmse:4.33898
[326]	Test-rmse:4.33898
[327]	Test-rmse:4.33898
[328]	Test-rmse:4.33898
[329]	Test-rmse:4.33898
[330]	Test-rmse:4.33898
[331]	Test-rmse:4.33898
[332]	Test-rmse:4.33898
[333]	Test-rmse:4.33898
[334]	Test-rmse:4.33898
[335]	Test-rmse:4.33898
[336]	Test-rmse:

[263]	Test-rmse:3.55805
[264]	Test-rmse:3.55805
[265]	Test-rmse:3.55804
[266]	Test-rmse:3.55804
[267]	Test-rmse:3.55804
[268]	Test-rmse:3.55804
[269]	Test-rmse:3.55804
[270]	Test-rmse:3.55803
[271]	Test-rmse:3.55803
[272]	Test-rmse:3.55803
[273]	Test-rmse:3.55803
[274]	Test-rmse:3.55803
[275]	Test-rmse:3.55803
[276]	Test-rmse:3.55803
[277]	Test-rmse:3.55802
[278]	Test-rmse:3.55802
[279]	Test-rmse:3.55802
[280]	Test-rmse:3.55802
[281]	Test-rmse:3.55802
[282]	Test-rmse:3.55802
[283]	Test-rmse:3.55802
[284]	Test-rmse:3.55801
[285]	Test-rmse:3.55801
[286]	Test-rmse:3.55801
[287]	Test-rmse:3.55801
[288]	Test-rmse:3.55801
[289]	Test-rmse:3.55800
[290]	Test-rmse:3.55800
[291]	Test-rmse:3.55800
[292]	Test-rmse:3.55800
[293]	Test-rmse:3.55800
[294]	Test-rmse:3.55800
[295]	Test-rmse:3.55800
[296]	Test-rmse:3.55800
[297]	Test-rmse:3.55799
[298]	Test-rmse:3.55799
[299]	Test-rmse:3.55799
[300]	Test-rmse:3.55799
[301]	Test-rmse:3.55799
[302]	Test-rmse:3.55799
[303]	Test-rmse:3.55799
[304]	Test-rmse:

[223]	Test-rmse:3.47005
[224]	Test-rmse:3.47004
[225]	Test-rmse:3.47004
[226]	Test-rmse:3.47003
[227]	Test-rmse:3.47002
[228]	Test-rmse:3.47001
[229]	Test-rmse:3.47000
[230]	Test-rmse:3.46999
[231]	Test-rmse:3.46999
[232]	Test-rmse:3.46998
[233]	Test-rmse:3.46998
[234]	Test-rmse:3.46997
[235]	Test-rmse:3.46996
[236]	Test-rmse:3.46996
[237]	Test-rmse:3.46995
[238]	Test-rmse:3.46994
[239]	Test-rmse:3.46994
[240]	Test-rmse:3.46993
[241]	Test-rmse:3.46993
[242]	Test-rmse:3.46992
[243]	Test-rmse:3.46992
[244]	Test-rmse:3.46991
[245]	Test-rmse:3.46991
[246]	Test-rmse:3.46990
[247]	Test-rmse:3.46990
[248]	Test-rmse:3.46990
[249]	Test-rmse:3.46989
[250]	Test-rmse:3.46989
[251]	Test-rmse:3.46988
[252]	Test-rmse:3.46988
[253]	Test-rmse:3.46988
[254]	Test-rmse:3.46987
[255]	Test-rmse:3.46987
[256]	Test-rmse:3.46987
[257]	Test-rmse:3.46987
[258]	Test-rmse:3.46986
[259]	Test-rmse:3.46986
[260]	Test-rmse:3.46986
[261]	Test-rmse:3.46986
[262]	Test-rmse:3.46985
[263]	Test-rmse:3.46985
[264]	Test-rmse:

In [22]:
# generate prediction file 
all_preds = pd.concat(predictions)


In [23]:
all_preds.reset_index(inplace=True)

In [24]:
all_preds = all_preds.reindex(ssubm.index)


In [25]:
all_preds.to_csv('submission.csv', index=False)

In [26]:
all_preds

,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,93.906807,101.061501
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,92.652992,101.127838
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,91.546089,101.899506
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,86.843857,101.246887
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,90.731544,109.282494
...,...,...,...,...
10128,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,213.668457,92.306480
10129,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,214.689362,97.443092
10130,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,207.836517,110.582932
10131,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,204.220551,115.576920
